# Supply chain partitioning example


In this notebook, we will use cuGraph to prototype partitioning of JDA supply chain example graph  

* Created:   11/3/2019
* Last Edit: 11/5/2019

RAPIDS Versions: 0.10.0

Test Hardware
* GV100 32G, CUDA 10.0


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon. 

### Test Data
We will be using the example graph provided:


![supply chain partitioning](../img/SC.png)


### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np

### Read data using cuDF

In [37]:
# Test file    
datafile='../data/supplyChain-data.csv'

In [38]:
# read the data using cuDF
gdf = cudf.read_csv(datafile, delimiter=" ", names=['src', 'dst'], dtype=['int32', 'int32'] )

### Adjust the vertex ID
Let's adjust all the vertex IDs to be zero based

In [39]:
gdf["src"] = gdf["src"] - 1
gdf["dst"] = gdf["dst"] - 1

In [40]:
# We are going to try spectral clustering algorithm which requires that there are edge weights. 
# In this case all the weights are being set to 1
gdf["data"] = cudf.Series(np.ones(len(gdf), dtype=np.float32))

In [41]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

,src,dst,data
0,0,5,1.0
1,0,6,1.0
2,0,7,1.0
3,0,8,1.0
4,1,10,1.0


### Create the graph

In [42]:
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"], gdf["data"])
#G.add_edge_list(gdf["src"], gdf["dst"])

In [43]:
print("Main Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

Main Graph
	Number of Vertices: 21
	Number of Edges:    26


In [44]:
G.degrees().head()

,vertex,in_degree,out_degree
0,0,0,4
1,1,0,3
2,2,0,2
3,3,0,2
4,4,0,3


### Now compute the Core Number

Core Number computes the core number for every vertex of a graph G. A k-core of a graph is a maximal subgraph that contains nodes of degree k or more. A node has a core number of k if it belongs to a k-core but not to k+1-core. This call does not support a graph with self-loops and parallel edges.

In [45]:
df = cugraph.core_number(G) 
df.head()

,vertex,core_number
0,0,4
1,1,3
2,2,2
3,3,2
4,4,3


### Remove vertices with largest out degree

In [47]:
#df.sort_values(by='core_number', ascending=False)
df1 = df[ df['core_number'] != df['core_number'].max()]
sg1 = cugraph.subgraph(G, df1['vertex'])

In [93]:
# Trial-and-error... more cutting keep on removing vertices with largest out degree
df2 = df1[ df1['core_number'] != df1['core_number'].max()]

sg2 = cugraph.subgraph(G, df2['vertex'])
print(df2['vertex'].count())

df3 = df2[ df2['core_number'] != df2['core_number'].max()]

sg3 = cugraph.subgraph(G, df3['vertex'])
print(df3['vertex'].count())
#print(sg3.number_of_nodes())

18
16


In [57]:
# try weakly CC after removing largest out-degree vertices
src, dst, val = sg1.view_edge_list()
src1, dst1 = cugraph.symmetrize(src, dst)
src1
G1 = cugraph.Graph()
G1.add_edge_list(src1, dst1)
df1 = cugraph.weakly_connected_components(G1)
df1

,labels,vertices
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
5,1,5
6,1,6
7,1,7
8,1,8
9,1,9


### Try Spectral Clustering -- balanced cut

#### Define and print function, but adjust vertex ID so that they match the illustration

In [14]:
def print_cluster(_df, id):
    
    _f = _df.query('cluster == @id')
  
    part = []
    for i in range(len(_f)):
        part.append(_f['vertex'][i] + 1)
    print(part)

In [17]:
# Call spectralBalancedCutClustering on the graph for 2 clusters
bc_gdf = cugraph.spectralBalancedCutClustering(sg1, 2, num_eigen_vects=2)

In [18]:
# See which nodes are in cluster 0:
print_cluster(bc_gdf, 0)

[2, 3, 4, 7, 13, 14, 16, 17, 18, 20]


In [19]:
print_cluster(bc_gdf, 1)

[1, 5, 6, 8, 9, 10, 11, 12, 15, 19]
